#  Неробастный и робастный t-тест: значимость коэффициентов

In [8]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_params, summary_col # вывод результатов тестирования
from scipy.stats import t # t-распределение и критические значения

# Не показывать FutureWarnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [9]:
# импорт данных
df = pd.read_csv('Labour.csv')
df.head()

,capital,labour,output,wage
0,2.606563,184,9.250759,43.080307
1,1.323237,91,3.664310,27.780016
2,22.093692,426,28.781516,44.467748
3,10.737851,72,4.124642,39.734710
4,1.161365,46,2.890150,34.650709


Рассмотрим регрессию

$$
	\log(output)=\beta_0+\beta_1\log(capital)+\beta_2\log(labour)+\beta_3\log^2(capital)+\beta_4\log^2(labour)+error
$$

Подгоним модель методом OLS с двумя опциями
* с оценкой ковариационной матрицы коэффициентов по умолчанию (неробастной) $Var_{OLS}(\hat{\beta})=s^2(X^\top X)^{-1}$, где $s^2=\sum e^2_i/(n-k-1)$
* с оценкой ковариационной матрицы коэффициентов, робастной к гетероскедастичности (HC3-оценкой)

Далее CME = Covariance Matrix Estimator

Стандартные ошибки коэффициентов $s.e.(\beta)$ - корни из диагональных элементов матрицы CME

In [ ]:
# спецификация модели
mod = smf.ols(formula='np.log(output)~np.log(capital)+np.log(labour)+I(np.log(capital)**2)+I(np.log(labour)**2)', data=df)
# подгонка модели с ковариационной матрицей по умолчанию (неробастной)
res_ols = mod.fit()
# подгонка модели с робастной HC3 ковариационной матрицей
res_hc = mod.fit(cov_type='HC3')

## Гипотеза
Тестируется гипотеза $H_0:\beta_{j}=0, \quad (j=1, \dots, 4)$

Пусть уровень значимости $\alpha=5\%=0.05$

Критическое значение t-распределения

In [11]:
# уровнь значимости
sign_level = 0.05
# критическое значение t-распределения
t.ppf(q=1-sign_level/2, df=mod.df_resid)

1.9641790265687167

## Результаты неробастного t-теста на значимость коэффициентов

In [12]:
summary_params(res_ols, alpha=0.05).round(3)

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
Intercept,-1.304,0.189,-6.914,0.000,-1.674,-0.934
np.log(capital),0.183,0.017,11.055,0.000,0.151,0.216
np.log(labour),0.515,0.083,6.181,0.000,0.352,0.679
I(np.log(capital) ** 2),0.023,0.005,4.518,0.000,0.013,0.033
I(np.log(labour) ** 2),0.020,0.010,2.112,0.035,0.001,0.039


Сравниваем модуль т-статистики (t) с критическим значением
P>t <0,05:
**ВЫВОД**: На уровне значимости 5% значимы коэффициенты при регрессорах:
* $\log(capital)$
* $\log(labour)$
* $log^2(capital)$
* $log^2(labour)$

## Результаты робастного t-теста на значимость коэффициентов (HC3-оценка ковариационной матрицы)

In [13]:
summary_params(res_hc, alpha=0.05).round(3)

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
Intercept,-1.304,0.493,-2.643,0.008,-2.271,-0.337
np.log(capital),0.183,0.029,6.215,0.000,0.125,0.241
np.log(labour),0.515,0.206,2.497,0.013,0.111,0.920
I(np.log(capital) ** 2),0.023,0.008,2.737,0.006,0.006,0.039
I(np.log(labour) ** 2),0.020,0.021,0.965,0.334,-0.021,0.061


**ВЫВОД**: На уровне значимости 5% значимы коэффициенты при регрессорах:
* $\log(capital)$
* $\log(labour)$
* $log^2(capital)$

## Результаты двух оцениваний в одной таблице

__Обратим внимание на коды значимости (количество звёздочек у коэффициентов)__

In [14]:
summary_col([res_ols, res_hc], model_names=['OLS-CME', 'HC3-CME'], stars=True)

,OLS-CME,HC3-CME
Intercept,-1.3039***,-1.3039***
,(0.1886),(0.4933)
np.log(capital),0.1831***,0.1831***
,(0.0166),(0.0295)
np.log(labour),0.5153***,0.5153**
,(0.0834),(0.2064)
I(np.log(capital) ** 2),0.0227***,0.0227***
,(0.0050),(0.0083)
I(np.log(labour) ** 2),0.0203**,0.0203
,(0.0096),(0.0210)
